# Install packages

In [ ]:
!pip install 'tensorflow[and-cuda]'
!pip install tensorflow-datasets
!pip install matplotlib
!pip install scikit-image

  Using cached tensorflow-2.20.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.5 kB)
  Using cached absl_py-2.4.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.12.19-py2.py3-none-any.whl.metadata (1.0 kB)
  Using cached gast-0.7.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-6.33.5-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached termcolor-3.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached wrapt-2.0.1-cp312-cp312-manylinux1_x86_64.manylinux_2_28_x86_64.manylinux_2_5_x86_64.whl.metadata (9.0 kB)
  Using cached grpcio-1.76.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (3.7 kB)
  Using cach

# Imports

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
from skimage import color
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


2026-01-31 10:29:05.173223: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2026-01-31 10:29:05.200643: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-01-31 10:29:05.791628: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Download dataset

In [2]:
DATASET_NAME = "places365_small"

In [3]:
if DATASET_NAME in tfds.list_builders():
    print("Dataset is in the list")
else:
    raise Exception("Dataset is not found in the list")

2026-01-31 10:29:14.948740: W external/local_xla/xla/tsl/platform/cloud/google_auth_provider.cc:185] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "ABORTED: All 10 retry attempts failed. The last failure: Error executing an HTTP request: HTTP response code 503 with body '<!DOCTYPE html PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html><head>
<meta type="copyright" content="Copyright (C) 1996-2021 The Squid Software Foundation and contributors">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
<title>FOUT: Die aangevraagde URL kon nie verkry word nie</title>
<style type="text/css"><!-- 
 /*
 * Copyright (C) 1996-2021 The Squid Software Foundation and contributors
 *
 * Squid software is distributed under GPLv2+ l'".


Dataset is in the list


In [ ]:
ds, info = tfds.load(DATASET_NAME,
               split='train[10%:]',
               download=True,
               data_dir="data"
)

/home/ai/anaconda3/envs/places/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dl Completed...: 0 url [00:00, ? url/s]
Dl Size...:  32%|███▏      | 9578/29911 [16:46<1:47:23,  3.16 MiB/s]

# LAB conversion function

In [ ]:

def rgb_to_lab_tf(image):
    def _rgb_to_lab(img):
        img = img.astype(np.float32) / 255.0
        lab = color.rgb2lab(img)
        return lab.astype(np.float32)

    lab = tf.numpy_function(
        _rgb_to_lab,
        [image],
        Tout=tf.float32
    )

    lab.set_shape([None, None, 3])
    return lab


## Apply conversion to dataset

In [ ]:
def preprocess(sample):
    image = sample["image"]              # uint8 RGB
    lab = rgb_to_lab_tf(image)            # LAB float32

    L = lab[..., :1]                      # Lightness
    AB = lab[..., 1:]                     # Color channels

    return L, AB                          # perfect for colorization


In [ ]:
dataset = dataset.map(
    preprocess,
    num_parallel_calls=tf.data.AUTOTUNE
)


## Optimize pipeline

In [ ]:
dataset = (
    dataset
    .shuffle(1000)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)


Now your dataset yields:

L  -> model input
AB -> model target

## Sanity-check the data

for L, AB in dataset.take(1):
    print("L shape:", L.shape)
    print("AB shape:", AB.shape)
    print("L range:", tf.reduce_min(L).numpy(), tf.reduce_max(L).numpy())
    print("AB range:", tf.reduce_min(AB).numpy(), tf.reduce_max(AB).numpy())


## Normalize

def normalize(L, AB):
    L = L / 100.0          # [0,1]
    AB = AB / 128.0        # [-1,1]
    return L, AB

dataset = dataset.map(
    normalize,
    num_parallel_calls=tf.data.AUTOTUNE
)


In [ ]:
early_stop = EarlyStopping(
    monitor="loss",        # or "val_loss" (better)
    patience=5,            # wait 5 epochs before stopping
    restore_best_weights=True
)


# Save the best model automatically

In [ ]:
checkpoint = ModelCheckpoint(
    "best_color_model.keras",
    monitor="loss",
    save_best_only=True
)


# Build a baseline model

In [ ]:

def build_model():
    inputs = layers.Input(shape=(None, None, 1))

    x = layers.Conv2D(64, 3, padding="same", activation="relu")(inputs)
    x = layers.Conv2D(64, 3, padding="same", activation="relu")(x)
    x = layers.Conv2D(2, 1, padding="same", activation="tanh")(x)

    return models.Model(inputs, x)

model = build_model()
model.summary()


# Compile the model

In [ ]:
model.compile(
    optimizer="adam",
    loss="mse"
)


# Train with stop control

In [ ]:
model.fit(
    dataset,
    epochs=100,            # upper bound, not a promise
    callbacks=[early_stop, checkpoint]
)


# Go with this 

# Split dataset:
train_ds = dataset.take(80)
val_ds = dataset.skip(80).take(20)


In [ ]:
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)

model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stop, checkpoint]
)


## Convert prediction back to RGB

In [ ]:
from skimage import color
import numpy as np

def lab_to_rgb(L, AB):
    L = L * 100.0
    AB = AB * 128.0
    lab = tf.concat([L, AB], axis=-1).numpy()
    rgb = color.lab2rgb(lab)
    return rgb


test

In [ ]:
for L, AB in dataset.take(1):
    pred_AB = model.predict(L)
    rgb = lab_to_rgb(L[0], pred_AB[0])


Proper dataset splitting (train / val / test correctly, not ad-hoc)

Image resizing policy (models require consistent spatial behavior)

Data augmentation (otherwise colors collapse / overfit)

Validation-based early stopping (not training loss)

Evaluation metrics (ΔE, SSIM) — otherwise you’re blind

Prediction visualization callback (to catch silent failure)

Robust inference pipeline (single image & batch)

Model upgrade from baseline CNN → U-Net (non-GAN)

# 🔹 STEP 11: Dataset split 

In [ ]:
ds = tfds.load("your_dataset", split="train", as_supervised=False)

TOTAL = 100  # replace with tfds info if known
train_ds = ds.take(int(0.8 * TOTAL))
val_ds   = ds.skip(int(0.8 * TOTAL)).take(int(0.1 * TOTAL))
test_ds  = ds.skip(int(0.9 * TOTAL))


# 🔹 STEP 12: Resize + preprocess

In [ ]:
IMG_SIZE = 256

def preprocess(sample):
    image = tf.image.resize(sample["image"], (IMG_SIZE, IMG_SIZE))
    lab = rgb_to_lab_tf(image)

    L  = lab[..., :1] / 100.0
    AB = lab[..., 1:] / 128.0

    return L, AB


# 🔹 STEP 13: Data augmentation (L channel only)

In [ ]:
def augment(L, AB):
    L = tf.image.random_flip_left_right(L)
    L = tf.image.random_brightness(L, 0.05)
    return L, AB


# 🔹 STEP 14: Final pipeline

In [ ]:
train_ds = (
    train_ds
    .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    .map(augment, num_parallel_calls=tf.data.AUTOTUNE)
    .shuffle(1000)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)

val_ds = (
    val_ds
    .map(preprocess)
    .batch(16)
    .prefetch(tf.data.AUTOTUNE)
)


# 🔹 STEP 15: U-Net (NON-GAN, STRONG BASELINE)

In [ ]:
from tensorflow.keras import layers, models

def unet():
    inputs = layers.Input((256, 256, 1))

    c1 = layers.Conv2D(64, 3, activation="relu", padding="same")(inputs)
    p1 = layers.MaxPool2D()(c1)

    c2 = layers.Conv2D(128, 3, activation="relu", padding="same")(p1)
    p2 = layers.MaxPool2D()(c2)

    b = layers.Conv2D(256, 3, activation="relu", padding="same")(p2)

    u1 = layers.UpSampling2D()(b)
    u1 = layers.Concatenate()([u1, c2])
    c3 = layers.Conv2D(128, 3, activation="relu", padding="same")(u1)

    u2 = layers.UpSampling2D()(c3)
    u2 = layers.Concatenate()([u2, c1])
    c4 = layers.Conv2D(64, 3, activation="relu", padding="same")(u2)

    outputs = layers.Conv2D(2, 1, activation="tanh")(c4)
    return models.Model(inputs, outputs)


# 🔹 STEP 16: Compile + proper early stopping

In [ ]:
model = unet()

model.compile(
    optimizer="adam",
    loss="mse"
)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=5,
    restore_best_weights=True
)


# 🔹 STEP 17: Training (FINAL)

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[early_stop]
)


# 🔹 STEP 18: ΔE metric (evaluation)

In [ ]:
import numpy as np
from skimage.color import deltaE_cie76

def delta_e(L, AB_pred, AB_true):
    lab_p = np.concatenate([L*100, AB_pred*128], axis=-1)
    lab_t = np.concatenate([L*100, AB_true*128], axis=-1)
    return deltaE_cie76(lab_t, lab_p).mean()


# 🔹 STEP 19: Inference pipeline (FINAL)

In [ ]:
from skimage.color import lab2rgb

def colorize(model, rgb_image):
    img = tf.image.resize(rgb_image, (256,256))
    lab = rgb_to_lab_tf(img)

    L = lab[..., :1] / 100.0
    AB = model.predict(L[None])[0] * 128.0

    lab_out = tf.concat([L*100, AB], axis=-1).numpy()
    return lab2rgb(lab_out)


# PART 2 — COMPLETE GAN IMPLEMENTATION (STEP-BY-STEP)

# 🔹 STEP 1: Generator (reuse your U-Net)

In [ ]:
def build_generator():
    return unet()  # your existing U-Net


# 🔹 STEP 2: Discriminator (PatchGAN)

In [ ]:
from tensorflow.keras import layers, models

def build_discriminator():
    inp = layers.Input((256, 256, 1))   # L
    tar = layers.Input((256, 256, 2))   # AB
    x = layers.Concatenate()([inp, tar])

    x = layers.Conv2D(64, 4, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(128, 4, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(256, 4, strides=2, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)

    x = layers.Conv2D(1, 4, padding="same")(x)

    return models.Model([inp, tar], x)


# 🔹 STEP 3: Loss functions (CRITICAL)

In [ ]:
loss_obj = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real, fake):
    real_loss = loss_obj(tf.ones_like(real)*0.9, real)  # label smoothing
    fake_loss = loss_obj(tf.zeros_like(fake), fake)
    return real_loss + fake_loss

def generator_loss(fake):
    return loss_obj(tf.ones_like(fake), fake)

def l1_loss(real, pred):
    return tf.reduce_mean(tf.abs(real - pred))


# 🔹 STEP 4: Optimizers (GAN-safe)

In [ ]:
gen_opt = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_opt = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)


# 🔹 STEP 5: GAN training step (THIS IS THE CORE)

In [ ]:
@tf.function
def train_step(L, AB):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_AB = generator(L, training=True)

        real_out = discriminator([L, AB], training=True)
        fake_out = discriminator([L, fake_AB], training=True)

        gen_adv = generator_loss(fake_out)
        gen_l1 = l1_loss(AB, fake_AB)
        gen_total = gen_adv + 100 * gen_l1

        disc_loss = discriminator_loss(real_out, fake_out)

    gen_grads = gen_tape.gradient(gen_total, generator.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_opt.apply_gradients(zip(gen_grads, generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grads, discriminator.trainable_variables))

    return gen_total, disc_loss


# 🔹 STEP 6: Training loop

In [ ]:
generator = build_generator()
discriminator = build_discriminator()

for epoch in range(50):
    for L, AB in train_ds:
        g_loss, d_loss = train_step(L, AB)

    print(f"Epoch {epoch}: G={g_loss:.3f}, D={d_loss:.3f}")
